# Importing neccesary library

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
!pip install biopython
import Bio  # Import the Biopython library
from Bio import SeqIO  # Import the SeqIO module for sequence manipulation
from Bio import AlignIO

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.9 MB/s eta 0:00:00


Installing HISAT2, SAMTOOLS, SUBREAD- for Read alignment, Feature count

In [3]:
# Install hisat2, samtools, and subread
!apt-get -qq install -y hisat2
!apt-get -qq install -y samtools
!apt-get -qq install -y subread



Selecting previously unselected package libhtscodecs2:amd64.
(Reading database ... 121671 files and directories currently installed.)
Preparing to unpack .../0-libhtscodecs2_1.1.1-3_amd64.deb ...
Unpacking libhtscodecs2:amd64 (1.1.1-3) ...
Selecting previously unselected package libhts3:amd64.
Preparing to unpack .../1-libhts3_1.13+ds-2build1_amd64.deb ...
Unpacking libhts3:amd64 (1.13+ds-2build1) ...
Selecting previously unselected package bcftools.
Preparing to unpack .../2-bcftools_1.13-1_amd64.deb ...
Unpacking bcftools (1.13-1) ...
Selecting previously unselected package hisat2.
Preparing to unpack .../3-hisat2_2.2.1-3_amd64.deb ...
Unpacking hisat2 (2.2.1-3) ...
Selecting previously unselected package python3-hisat2.
Preparing to unpack .../4-python3-hisat2_2.2.1-3_all.deb ...
Unpacking python3-hisat2 (2.2.1-3) ...
Selecting previously unselected package samtools.
Preparing to unpack .../5-samtools_1.13-4_amd64.deb ...
Unpacking samtools (1.13-4) ...
Setting up libhtscodecs2:amd6

Downloading of Reference genome

In [11]:
!wget https://genome-idx.s3.amazonaws.com/hisat/grch38_genome.tar.gz
!tar -xzvf grch38_genome.tar.gz

--2024-01-28 14:55:08--  https://genome-idx.s3.amazonaws.com/hisat/grch38_genome.tar.gz
Resolving genome-idx.s3.amazonaws.com (genome-idx.s3.amazonaws.com)... 3.5.9.100, 54.231.131.153, 16.182.32.113, ...
Connecting to genome-idx.s3.amazonaws.com (genome-idx.s3.amazonaws.com)|3.5.9.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4210306865 (3.9G) [binary/octet-stream]
Saving to: ‘grch38_genome.tar.gz’

grch38_genome.tar.g 100%[===================>]   3.92G  25.2MB/s    in 2m 6s   

2024-01-28 14:57:14 (32.0 MB/s) - ‘grch38_genome.tar.gz’ saved [4210306865/4210306865]

grch38/
grch38/genome.5.ht2
grch38/genome.2.ht2
grch38/make_grch38.sh
grch38/genome.3.ht2
grch38/genome.4.ht2
grch38/genome.7.ht2
grch38/genome.1.ht2
grch38/genome.6.ht2
grch38/genome.8.ht2


Read alignment with reference genom

In [16]:
!hisat2 -x grch38/genome -U demo_trimmed.fastq -S aligned_reads.sam



1249800 reads; of these:
  1249800 (100.00%) were unpaired; of these:
    86563 (6.93%) aligned 0 times
    1082380 (86.60%) aligned exactly 1 time
    80857 (6.47%) aligned >1 times
93.07% overall alignment rate


In [17]:
!samtools view -bS aligned_reads.sam > aligned_reads.bam


In [18]:
!samtools sort aligned_reads.bam -o sorted_aligned_reads.bam

In [20]:
!samtools index sorted_aligned_reads.bam

HISAT Feature count

In [21]:
!wget https://ftp.ensembl.org/pub/release-111/gtf/homo_sapiens/Homo_sapiens.GRCh38.111.gtf.gz

--2024-01-28 15:36:34--  https://ftp.ensembl.org/pub/release-111/gtf/homo_sapiens/Homo_sapiens.GRCh38.111.gtf.gz
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54350257 (52M) [application/x-gzip]
Saving to: ‘Homo_sapiens.GRCh38.111.gtf.gz’

Homo_sapiens.GRCh38 100%[===================>]  51.83M   458KB/s    in 1m 58s  

2024-01-28 15:38:32 (452 KB/s) - ‘Homo_sapiens.GRCh38.111.gtf.gz’ saved [54350257/54350257]



In [22]:
!gunzip Homo_sapiens.GRCh38.111.gtf.gz

In [23]:
!apt-get install subread


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
subread is already the newest version (2.0.3+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [25]:
!featureCounts -a Homo_sapiens.GRCh38.111.gtf -o counts.txt -T 4 sorted_aligned_reads.bam



        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.3

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 1 BAM file                                       ||
||                                                                            ||
||                           sorted_aligned_reads.bam                         ||
||                                                                            ||
||             Output file : counts.txt                          

# Data Analysis of The Gene count

importing the feature count into dataframe

In [27]:
counts_df = pd.read_csv('counts.txt', sep='\t', comment='#', index_col=0)

# Display the DataFrame
counts_df

,Chr,Start,End,Strand,Length,sorted_aligned_reads.bam
Geneid,,,,,,
ENSG00000279928,1;1;1;1;1,182696;183132;183494;183740;183981,182746;183216;183571;183901;184174,+;+;+;+;+,570,0
ENSG00000228037,1;1;1,2581560;2583370;2584125,2581650;2583495;2584533,+;+;+,626,0
ENSG00000142611,1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;...,3069168;3069183;3069197;3069203;3069211;318612...,3069296;3069296;3069296;3069296;3069296;318647...,+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;...,9767,32
ENSG00000284616,1;1;1;1,5301928;5303328;5304401;5306942,5302004;5303393;5305029;5307394,-;-;-;-,1225,0
ENSG00000157911,1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;1;...,2403964;2403974;2404061;2404613;2404838;240503...,2405834;2405834;2404371;2404891;2405828;240583...,-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;...,3782,22
...,...,...,...,...,...,...
ENSG00000271254,KI270711.1;KI270711.1;KI270711.1;KI270711.1;KI...,4612;6101;6991;6991;7817;7817;8604;8604;9365;9...,6370;6370;7102;7102;7989;7989;8655;8655;9537;9...,-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;...,4292,12
ENSG00000275987,KI270713.1,30437,30580,-,144,1
ENSG00000268674,KI270713.1,35407,35916,+,510,0


In [28]:
counts_df.describe()

,Length,sorted_aligned_reads.bam
count,63241.000000,63241.000000
mean,2598.993786,13.868139
std,4099.111299,76.772862
min,8.000000,0.000000
25%,430.000000,0.000000
50%,1083.000000,0.000000
75%,3379.000000,1.000000
max,347964.000000,7633.000000


In [29]:
counts_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63241 entries, ENSG00000279928 to ENSG00000275405
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Chr                       63241 non-null  object
 1   Start                     63241 non-null  object
 2   End                       63241 non-null  object
 3   Strand                    63241 non-null  object
 4   Length                    63241 non-null  int64 
 5   sorted_aligned_reads.bam  63241 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 5.4+ MB


Check Next file for Subsequent Data Analysis